In [60]:
import cv2

In [61]:
import numpy as np 

In [62]:
import matplotlib.pyplot as plt

In [63]:
face_data = cv2.CascadeClassifier('face.xml')

In [64]:
with_mask = np.load('with_mask.npy')

In [65]:
without_mask = np.load('without_mask.npy')

In [66]:
with_mask.shape

(200, 50, 50, 3)

In [67]:
without_mask.shape

(200, 50, 50, 3)

In [68]:
with_mask = with_mask.reshape(200,50*50*3)
without_mask = without_mask.reshape(200,50*50*3)

In [69]:
with_mask.shape

(200, 7500)

In [70]:
without_mask.shape

(200, 7500)

In [71]:
X=np.r_[with_mask,without_mask]

In [72]:
X.shape

(400, 7500)

In [73]:
Y = np.zeros(X.shape[0])

In [74]:
Y[200:] = 1.0

In [75]:
names = {0: 'Mask', 1: 'NoMask'}

In [76]:
#svm - support vector Machine
#SVC - support vector Classification
#PCA - Principle component Analysic
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [89]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.25)

In [94]:
x_train.shape

(320, 7500)

In [95]:
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [96]:
x_train[0]

array([-3210.25250995,   790.52747695,   703.36472152])

In [93]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20)

In [97]:
SVC()

SVC()

In [98]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [99]:
x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [100]:
accuracy_score(y_test, y_pred)

0.9875

In [101]:
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = face_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
            face = img [y:y+h, x:x+h, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            pred = svm.predict(face)[0]
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (244,250,250),2)
        cv2.imshow('result',img)
        if cv2.waitKey(120) == 27:
            break
capture.release()
cv2.destroyAllWindows()